In [ ]:
#!pip install cx_Oracle

In [1]:
import cx_Oracle as ora

In [2]:
conn = ora.connect("kosmorpa/test00@192.168.0.68:1521/orcl")
conn.version

'11.2.0.1.0'

In [ ]:
# SQL Developer에서 작성 후 실행하기
'''
create table board2(
    no number constraint board_no_pk primary key,
    pwd varchar2(10),
    writer varchar2(34),
    subject varchar2(100),
    content varchar2(400),
    regdate date default sysdate
);

create sequence board2_seq
increment by 1
start with 1;

commit;
'''

In [3]:
# Oracle DB 접속한 객체의 주소값 cursor()
cursor = conn.cursor()
print(cursor)

<cx_Oracle.Cursor on <cx_Oracle.Connection to kosmorpa@192.168.0.68:1521/orcl>>


In [ ]:
# 입력처리
writerv = input("작성자 :")
pwdv = input("비밀번호를 입력 :")
subjectv = input("제목 :")
contentv = input("내용 :")
print("\n")
print(writerv)
print(pwdv)
print(subjectv)
print(contentv)

In [ ]:
# SQL 문 작성
sql_insert='insert into board2 values(board2_seq.nextVal,:pwd,:writer,:subject,:content,sysdate)'
#print(sql_insert)
# 입력받은 값과 sql 바인딩 처리
# execute(sql,bind)
cursor.execute(sql_insert,pwd=pwdv,writer=writerv,subject=subjectv,content=contentv)
cursor.close()
conn.commit()
# 오라클과 연결 종료
conn.close()

In [ ]:
conn = ora.connect("kosmorpa/test00@192.168.0.68:1521/orcl")
conn.version

In [ ]:
# fetchall()
cursor = conn.cursor()
#print(cursor)
sql_select = 'select no,writer,subject,content,regdate from board2 order by 1 desc'
cursor.execute(sql_select)
print(cursor.fetchall())
print(type(cursor.fetchall()))
cursor.close()
conn.close()

In [ ]:
conn = ora.connect("kosmorpa/test00@192.168.0.68:1521/orcl")
conn.version

In [4]:
# findone() : mybatis에서 select_one()
# Flask 웹 서비스일 경우 링크를 통해서 파라미터가 전달
nov = input("상세보기 값 :")
sql_no = "select no,writer,subject,content, to_char(regdate, 'yyyy-mm-dd') \
regdate from board2 where no=:no"
cursor = conn.cursor()
#print(cursor)
cursor.execute(sql_no,no=nov)
print(cursor.fetchone())
cursor.close()

상세보기 값 :2
(2, '김길동', '하이', '내용', '2020-09-21')


In [5]:
import cx_Oracle as ora

def connections():
    try:
        conn = ora.connect("kosmorpa/test00@192.168.0.68:1521/orcl")
    except Exception as e:
        conn = "예외발생"
        print(e)
        return
    return conn

def test():
    print(connections())
    
test()

<cx_Oracle.Connection to kosmorpa@192.168.0.68:1521/orcl>


In [17]:
# 연습문제) 최근 글 5개만 출력하시오.
def examSelect():
    conn = connections()
    print(conn)
    sql_select = 'select no,writer,content,regdate from \
    (select rownum,no,writer,content,regdate from \
    (select * from board2 order by 1 desc) where rownum <= 5)'
    cursor = conn.cursor()
    cursor.execute(sql_select)
    rs = cursor.fetchall()
    #print(cursor.fetchall())
    #print(type(cursor.fetchall()))
    cursor.close()
    conn.close()
    return rs

def printoutvalues():
    sql_recent = "select * from board2 where rownum <=5 order by regdate desc;"
    cursor = conn.cursor()
    print(sql_recent)
    #cursor.execute(sql_recent)
    #print(cursor.fetchone())
    
selectlist5 = examSelect()
for e in selectlist5:
    print(e)

<cx_Oracle.Connection to kosmorpa@192.168.0.68:1521/orcl>
(6, '남주', '벚꽃이 떨어지는 속도', datetime.datetime(2020, 9, 21, 11, 44))
(5, '장첸', '그런것까지 알아야 되니?', datetime.datetime(2020, 9, 21, 11, 44, 6))
(4, '수양대군', '내 몸속에 일본놈들의 총알이 여섯 개나 박혀 있습니다', datetime.datetime(2020, 9, 21, 11, 43, 7))
(3, '둘리', '호잇호잇!!', datetime.datetime(2020, 9, 21, 11, 10, 50))
(2, '김길동', '내용', datetime.datetime(2020, 9, 21, 10, 49, 53))


In [16]:
def myFetchmany():
    conn = connections()
    cursor = conn.cursor()
    sql_select = 'select * from board2 order by 1 desc'
    cursor.execute(sql_select)
    numRows = 5
    res = cursor.fetchmany(numRows = numRows)
    print('fetchmany {}'.format(numRows))
    cursor.close()
    conn.close()
    return res

print(myFetchmany())

fetchmany 5
[(6, '11', '남주', '초속 5cm', '벚꽃이 떨어지는 속도', datetime.datetime(2020, 9, 21, 11, 44)), (5, '11', '장첸', '돈 받으러 왔는데', '그런것까지 알아야 되니?', datetime.datetime(2020, 9, 21, 11, 44, 6)), (4, '11', '수양대군', '내가 왕이 될 상인가', '내 몸속에 일본놈들의 총알이 여섯 개나 박혀 있습니다', datetime.datetime(2020, 9, 21, 11, 43, 7)), (3, '11', '둘리', '호잇!', '호잇호잇!!', datetime.datetime(2020, 9, 21, 11, 10, 50)), (2, '11', '김길동', '하이', '내용', datetime.datetime(2020, 9, 21, 10, 49, 53))]


In [19]:
# 삭제 - insert,delete,update
def boardDel(num):
    conn = connections()
    cursor = conn.cursor()
    sql_delete = 'delete from board2 where no = :no'
    try:
        cursor.execute(sql_delete, no=num)
        conn.commit()
        except Exception as e:
            print(e)
        finally:
            conn.close()
            
num = input('삭제번호:')
boardDel(num)
examSelect()

SyntaxError: invalid syntax (<ipython-input-19-aa052565b33d>, line 9)